In [36]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import json
import os

In [37]:
def round_hour(x):
    l=len(x)
    total_length = max(4, len(x))
    x = x.zfill(total_length)
    s=x[2]+x[3]
    s1=x[0]+x[1]
    ans=""
    if int(s)>=30:
        p=int(s1)+1
        ans=str(p)+"00"
    if len(ans)==0:
        ans=s1+"00"
    if ans=="2400":
        return "0000"
    return ans


In [3]:
dfs=[]
for year in os.listdir("./Flights/"):
    for month in os.listdir(f"./Flights/{year}"):
        for file in os.listdir(f"./Flights/{year}/{month}"):
            if file.endswith(".csv"):
                temp=pd.read_csv(f"./Flights/{year}/{month}/{file}", sep=",")
                temp=temp[["FlightDate", "Quarter" ,"Year", "Month", "DayofMonth", "DepTime", "DepDel15", "CRSDepTime", "DepDelayMinutes", "Origin", "Dest", "ArrTime", "CRSArrTime", "ArrDel15", "ArrDelayMinutes"]]
                dfs.append(temp)


C:\Users\Amit\AppData\Local\Temp\ipykernel_2136\2232973509.py:6: DtypeWarning: Columns (77) have mixed types. Specify dtype option on import or set low_memory=False.
  temp=pd.read_csv(f"./Flights/{year}/{month}/{file}", sep=",")
C:\Users\Amit\AppData\Local\Temp\ipykernel_2136\2232973509.py:6: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  temp=pd.read_csv(f"./Flights/{year}/{month}/{file}", sep=",")
C:\Users\Amit\AppData\Local\Temp\ipykernel_2136\2232973509.py:6: DtypeWarning: Columns (48,69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  temp=pd.read_csv(f"./Flights/{year}/{month}/{file}", sep=",")
C:\Users\Amit\AppData\Local\Temp\ipykernel_2136\2232973509.py:6: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  temp=pd.read_csv(f"./Flights/{year}/{month}/{file}", sep=",")
C:\Users\Amit\AppData\Local\Temp\ipykernel_2136\22329735

In [6]:
flights=pd.concat(dfs,ignore_index=True)

In [8]:
flights["ArrTime"]= flights["ArrTime"].fillna(0.0)

In [11]:
flights["RoundTime"]= flights["ArrTime"].apply(lambda x: int(round_hour(str(int(x)))))

In [12]:
flights=flights.rename(columns={"FlightDate": "date", "Dest" :"airport", "RoundTime" : "time"})

In [13]:
flights.to_csv("./flights.csv",sep=",")

In [18]:
hourly_data=[]
for airport in os.listdir("./Weather/"):
    for file in os.listdir(f"./Weather/{airport}"):
        if file.startswith("2016") or file.startswith("2017"):
            with open(f"./Weather/{airport}/{file}","r") as f:
                data=json.load(f)
                for i in data["data"]["weather"]:
                    for j in i["hourly"]:
                        data1=j
                        for item in ['mintempC', 'maxtempF', 'sunHour', 'mintempF', 'maxtempC', 'totalSnow_cm', 'date', 'astronomy', 'uvIndex']:
                            data1[item]=i[item]
                        data1["airport"]=airport
                        hourly_data.append(data1)


In [22]:
weather=pd.DataFrame(hourly_data)

In [24]:
weather=weather[["windspeedKmph", "winddirDegree", "weatherCode", "precipMM", "visibility", "pressure", "cloudcover", "DewPointF", "WindGustKmph", "tempF", "WindChillF", "humidity", "date", "time", "airport"]]

In [25]:
weather["time"]=weather["time"].apply( lambda x: int(x))

In [27]:
weather.to_csv("./weather.csv", sep=",")

In [31]:
merged_inner=pd.merge(flights, weather, on=["date", "airport", "time"], how="inner")

In [32]:
merged_inner.to_csv("./merged_inner.csv",sep=",")

In [33]:
merged_left=pd.merge(flights, weather, on=["date", "airport", "time"], how="left")

In [34]:
merged_left.to_csv("./merged_left.csv",sep=",")

In [38]:
flights

,date,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,Origin,airport,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,time
0,2016-01-06,1,2016,1,6,1057.0,0.0,1100,0.0,DFW,DTW,1432.0,1438,0.0,0.0,1500
1,2016-01-07,1,2016,1,7,1056.0,0.0,1100,0.0,DFW,DTW,1426.0,1438,0.0,0.0,1400
2,2016-01-08,1,2016,1,8,1055.0,0.0,1100,0.0,DFW,DTW,1445.0,1438,0.0,7.0,1500
3,2016-01-09,1,2016,1,9,1102.0,0.0,1100,2.0,DFW,DTW,1433.0,1438,0.0,0.0,1500
4,2016-01-10,1,2016,1,10,1240.0,1.0,1100,100.0,DFW,DTW,1631.0,1438,1.0,113.0,1700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11292274,2017-09-25,3,2017,9,25,1332.0,1.0,1018,194.0,ORD,BOS,1647.0,1335,1.0,192.0,1700
11292275,2017-09-25,3,2017,9,25,1822.0,0.0,1828,0.0,JFK,LGB,2135.0,2135,0.0,0.0,2200
11292276,2017-09-25,3,2017,9,25,623.0,0.0,630,0.0,JAX,FLL,732.0,743,0.0,0.0,800
11292277,2017-09-25,3,2017,9,25,654.0,0.0,700,0.0,FLL,JAX,802.0,812,0.0,0.0,800


In [39]:
weather

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport
0,11,318,176,0.1,10,1023,100,47,17,49,46,91,2016-01-01,0,ATL
1,13,317,176,0.0,10,1023,100,44,22,46,42,92,2016-01-01,100,ATL
2,14,315,122,0.0,10,1023,100,41,26,43,38,92,2016-01-01,200,ATL
3,16,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,300,ATL
4,17,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,400,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263155,26,296,113,0.0,10,1014,0,57,27,67,67,70,2017-09-30,1900,SFO
263156,26,298,113,0.0,10,1014,0,58,27,67,67,71,2017-09-30,2000,SFO
263157,26,300,113,0.0,10,1014,0,58,27,67,67,73,2017-09-30,2100,SFO
263158,23,302,113,0.0,10,1014,0,58,25,66,66,74,2017-09-30,2200,SFO


In [40]:
merged_inner

,date,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,Origin,...,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity
0,2016-01-01,1,2016,1,1,1539.0,1.0,1513,26.0,DTW,...,119,0.0,10,1033,50,32,15,48,43,57
1,2016-01-01,1,2016,1,1,1504.0,0.0,1509,0.0,ORF,...,119,0.0,10,1033,50,32,15,48,43,57
2,2016-01-01,1,2016,1,1,1630.0,1.0,1604,26.0,IAH,...,119,0.0,10,1033,50,32,15,48,43,57
3,2016-01-01,1,2016,1,1,1545.0,0.0,1533,12.0,DCA,...,119,0.0,10,1033,50,32,15,48,43,57
4,2016-01-01,1,2016,1,1,1516.0,0.0,1513,3.0,BWI,...,119,0.0,10,1033,50,32,15,48,43,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5055587,2017-09-23,3,2017,9,23,2211.0,1.0,2149,22.0,BOS,...,113,0.0,10,1009,6,61,7,67,67,81
5055588,2017-09-23,3,2017,9,23,1652.0,0.0,1640,12.0,LAX,...,113,0.0,10,1014,30,57,32,70,70,62
5055589,2017-09-23,3,2017,9,23,2151.0,1.0,2135,16.0,BOS,...,116,0.1,7,1012,47,72,22,75,75,90
5055590,2017-09-24,3,2017,9,24,2322.0,1.0,2135,107.0,BOS,...,113,0.0,10,1012,10,73,20,76,76,90


In [41]:
merged_left

,date,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,Origin,...,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity
0,2016-01-06,1,2016,1,6,1057.0,0.0,1100,0.0,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-07,1,2016,1,7,1056.0,0.0,1100,0.0,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-08,1,2016,1,8,1055.0,0.0,1100,0.0,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-09,1,2016,1,9,1102.0,0.0,1100,2.0,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-10,1,2016,1,10,1240.0,1.0,1100,100.0,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11292274,2017-09-25,3,2017,9,25,1332.0,1.0,1018,194.0,ORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11292275,2017-09-25,3,2017,9,25,1822.0,0.0,1828,0.0,JFK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11292276,2017-09-25,3,2017,9,25,623.0,0.0,630,0.0,JAX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11292277,2017-09-25,3,2017,9,25,654.0,0.0,700,0.0,FLL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
merged_inner.isna().sum()

date                   0
Quarter                0
Year                   0
Month                  0
DayofMonth             0
DepTime            64651
DepDel15           64668
CRSDepTime             0
DepDelayMinutes    64668
Origin                 0
airport                0
ArrTime                0
CRSArrTime             0
ArrDel15           78567
ArrDelayMinutes    78567
time                   0
windspeedKmph          0
winddirDegree          0
weatherCode            0
precipMM               0
visibility             0
pressure               0
cloudcover             0
DewPointF              0
WindGustKmph           0
tempF                  0
WindChillF             0
humidity               0
dtype: int64

In [82]:
merged_data=merged_inner.dropna()

In [83]:
merged_data=merged_data[merged_data["Origin"].isin(["ATL", "CLT", "DEN", "DFW", "EWR", "IAH", "JFK", "LAS", "LAX", "MCO", "MIA", "ORD", "PHX", "SEA", "SFO"])]

In [84]:
merged_data.to_csv("./merged_data.csv",sep=",")

In [50]:
from sklearn.model_selection import train_test_split

In [85]:
merged_data.dtypes

date                object
Quarter              int64
Year                 int64
Month                int64
DayofMonth           int64
DepTime            float64
DepDel15           float64
CRSDepTime           int64
DepDelayMinutes    float64
Origin              object
airport             object
ArrTime            float64
CRSArrTime           int64
ArrDel15           float64
ArrDelayMinutes    float64
time                 int64
windspeedKmph       object
winddirDegree       object
weatherCode         object
precipMM            object
visibility          object
pressure            object
cloudcover          object
DewPointF           object
WindGustKmph        object
tempF               object
WindChillF          object
humidity            object
dtype: object

In [97]:
merged_data["precipMM"]=merged_data["precipMM"].apply(lambda x: float(x))
merged_data["windspeedKmph"]=merged_data["windspeedKmph"].apply(lambda x: int(x))
merged_data["winddirDegree"]=merged_data["winddirDegree"].apply(lambda x: int(x))
merged_data["weatherCode"]=merged_data["weatherCode"].apply(lambda x: int(x))
merged_data["visibility"]=merged_data["visibility"].apply(lambda x: int(x))
merged_data["pressure"]=merged_data["pressure"].apply(lambda x: int(x))
merged_data["cloudcover"]=merged_data["cloudcover"].apply(lambda x: int(x))
merged_data["DewPointF"]=merged_data["DewPointF"].apply(lambda x: int(x))
merged_data["WindGustKmph"]=merged_data["WindGustKmph"].apply(lambda x: int(x))
merged_data["tempF"]=merged_data["tempF"].apply(lambda x: int(x))
merged_data["WindChillF"]=merged_data["WindChillF"].apply(lambda x: int(x))
merged_data["humidity"]=merged_data["humidity"].apply(lambda x: int(x))
merged_data["ArrDel15"]=merged_data["ArrDel15"].apply(lambda x: int(x))



In [98]:
merged_data.dtypes

date               datetime64[ns]
Quarter                     int64
Year                        int64
Month                       int64
DayofMonth                  int64
DepTime                   float64
DepDel15                  float64
CRSDepTime                  int64
DepDelayMinutes           float64
Origin                      int32
airport                     int32
ArrTime                   float64
CRSArrTime                  int64
ArrDel15                    int64
ArrDelayMinutes           float64
time                        int64
windspeedKmph               int64
winddirDegree               int64
weatherCode                 int64
precipMM                  float64
visibility                  int64
pressure                    int64
cloudcover                  int64
DewPointF                   int64
WindGustKmph                int64
tempF                       int64
WindChillF                  int64
humidity                    int64
dtype: object

In [90]:
merged_data["date"] =pd.to_datetime(merged_data["date"])

In [92]:
from sklearn.preprocessing import LabelEncoder

In [94]:
le=LabelEncoder()
le.fit(merged_data["airport"])

LabelEncoder()

In [95]:
merged_data["airport"]=le.transform(merged_data["airport"])
merged_data["Origin"]=le.transform(merged_data["Origin"])


In [100]:
X=merged_data.drop(["ArrDel15"],axis=1)
y=merged_data["ArrDel15"]

In [104]:
X=X.drop(["date"], axis=1)

In [105]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2)

In [110]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [107]:
dt=DecisionTreeClassifier()
dt.fit(X_train, y_train)
print(dt.score(X_test, y_test))

1.0


In [111]:
y_pred=dt.predict(X_test)
cm=confusion_matrix(y_test,y_pred,labels=dt.classes_)
cm

array([[292568,      0],
       [     0,  77720]], dtype=int64)

In [108]:
rf=RandomForestClassifier()
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))

1.0


In [112]:
y_pred=rf.predict(X_test)
cm=confusion_matrix(y_test,y_pred,labels=rf.classes_)
cm

array([[292568,      0],
       [     0,  77720]], dtype=int64)